In [1]:
import yfinance as yf
from datetime import datetime, timedelta

# Calculate 15 years ago from today
today = datetime.now()
fifteen_years_ago = today - timedelta(days=15*365.25)  # Account for leap years
yesterday = datetime.now() - timedelta(days=1)

data = yf.download('^GSPC', start=fifteen_years_ago, end=yesterday)

data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-10-05,1026.869995,1042.579956,1025.920044,1040.459961,1040.459961,4313310000
2009-10-06,1042.020020,1060.550049,1042.020020,1054.719971,1054.719971,5029840000
2009-10-07,1053.650024,1058.020020,1050.099976,1057.579956,1057.579956,4238220000
2009-10-08,1060.030029,1070.670044,1060.030029,1065.479980,1065.479980,4988400000
2009-10-09,1065.280029,1071.510010,1063.000000,1071.489990,1071.489990,3763780000


In [21]:
# Step 2: Train-test split (80% train, 20% test)
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

In [24]:
import pandas as pd
data.index = pd.to_datetime(data.index)
train_data.index = pd.to_datetime(train_data.index)
test_data.index = pd.to_datetime(test_data.index)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
test_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-10-01,4317.160156,4375.189941,4288.520020,4357.040039,4357.040039,4010370000
2021-10-04,4348.839844,4355.509766,4278.939941,4300.459961,4300.459961,4307870000
2021-10-05,4309.870117,4369.229980,4309.870117,4345.720215,4345.720215,3902890000
2021-10-06,4319.569824,4365.569824,4290.490234,4363.549805,4363.549805,4009630000
2021-10-07,4383.729980,4429.970215,4383.729980,4399.759766,4399.759766,3843740000
...,...,...,...,...,...,...
2024-09-26,5762.220215,5767.370117,5721.009766,5745.370117,5745.370117,4391180000
2024-09-27,5755.359863,5763.779785,5727.339844,5738.169922,5738.169922,3898530000
2024-09-30,5726.520020,5765.140137,5703.529785,5762.479980,5762.479980,4425730000


In [28]:
date_range = pd.date_range(start=data.index.min(), end=data.index.max())
missing_dates = date_range.difference(data.index)

test_date_range = pd.date_range(start=test_data.index.min(), end=test_data.index.max())
missing_test_dates = test_date_range.difference(test_data.index)

train_date_range = pd.date_range(start=train_data.index.min(), end=train_data.index.max())
missing_train_dates = train_date_range.difference(train_data.index)

print(f"Missing Dates: {missing_dates}")
print(f"Missing Test Dates: {missing_test_dates}")
print(f"Missing Train Dates: {missing_train_dates}")

Missing Dates: DatetimeIndex(['2009-10-10', '2009-10-11', '2009-10-17', '2009-10-18',
               '2009-10-24', '2009-10-25', '2009-10-31', '2009-11-01',
               '2009-11-07', '2009-11-08',
               ...
               '2024-09-01', '2024-09-02', '2024-09-07', '2024-09-08',
               '2024-09-14', '2024-09-15', '2024-09-21', '2024-09-22',
               '2024-09-28', '2024-09-29'],
              dtype='datetime64[ns]', length=1703, freq=None)
Missing Test Dates: DatetimeIndex(['2021-10-02', '2021-10-03', '2021-10-09', '2021-10-10',
               '2021-10-16', '2021-10-17', '2021-10-23', '2021-10-24',
               '2021-10-30', '2021-10-31',
               ...
               '2024-09-01', '2024-09-02', '2024-09-07', '2024-09-08',
               '2024-09-14', '2024-09-15', '2024-09-21', '2024-09-22',
               '2024-09-28', '2024-09-29'],
              dtype='datetime64[ns]', length=343, freq=None)
Missing Train Dates: DatetimeIndex(['2009-10-10', '2009-10-11'

In [29]:
# Reindex the DataFrame to include all the dates
data_full = data.reindex(date_range)

# Fill missing values with forward fill (propogating the last known value)
data_full.ffill(inplace=True)

missing_values = data_full.isnull().sum()
print(f"Missing values after forward fill: \n{missing_values}")

# Reindex the Train DataFrame to include all the dates
train_data_full = train_data.reindex(train_date_range)

# Fill missing values with forward fill (propogating the last known value)
train_data_full.ffill(inplace=True)

missing_values = train_data_full.isnull().sum()
print(f"Missing values after forward fill: \n{missing_values}")

# Reindex the Train DataFrame to include all the dates
test_data_full = test_data.reindex(test_date_range)

# Fill missing values with forward fill (propogating the last known value)
test_data_full.ffill(inplace=True)

missing_values = test_data_full.isnull().sum()
print(f"Missing values after forward fill: \n{missing_values}")

Missing values after forward fill: 
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
Missing values after forward fill: 
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
Missing values after forward fill: 
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [30]:
date_range = pd.date_range(start=data_full.index.min(), end=data_full.index.max())
missing_dates = date_range.difference(data_full.index)
print(f"Missing Dates: {missing_dates}")

train_date_range = pd.date_range(start=train_data_full.index.min(), end=train_data_full.index.max())
missing_dates = train_date_range.difference(train_data_full.index)
print(f"Missing Dates: {missing_dates}")

test_date_range = pd.date_range(start=test_data_full.index.min(), end=test_data_full.index.max())
missing_dates = test_date_range.difference(test_data_full.index)
print(f"Missing Dates: {missing_dates}")

Missing Dates: DatetimeIndex([], dtype='datetime64[ns]', freq='D')
Missing Dates: DatetimeIndex([], dtype='datetime64[ns]', freq='D')
Missing Dates: DatetimeIndex([], dtype='datetime64[ns]', freq='D')


In [6]:
data_full.head(30)

,Open,High,Low,Close,Adj Close,Volume
2009-10-05,1026.869995,1042.579956,1025.920044,1040.459961,1040.459961,4.313310e+09
2009-10-06,1042.020020,1060.550049,1042.020020,1054.719971,1054.719971,5.029840e+09
2009-10-07,1053.650024,1058.020020,1050.099976,1057.579956,1057.579956,4.238220e+09
2009-10-08,1060.030029,1070.670044,1060.030029,1065.479980,1065.479980,4.988400e+09
2009-10-09,1065.280029,1071.510010,1063.000000,1071.489990,1071.489990,3.763780e+09
2009-10-10,1065.280029,1071.510010,1063.000000,1071.489990,1071.489990,3.763780e+09
2009-10-11,1065.280029,1071.510010,1063.000000,1071.489990,1071.489990,3.763780e+09
2009-10-12,1071.630005,1079.459961,1071.630005,1076.189941,1076.189941,3.710430e+09
2009-10-13,1074.959961,1075.300049,1066.709961,1073.189941,1073.189941,4.320480e+09
2009-10-14,1078.680054,1093.170044,1078.680054,1092.020020,1092.020020,5.406420e+09


In [31]:
train_data_full.head(30)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Open,High,Low,Close,Adj Close,Volume
2009-10-05,1026.869995,1042.579956,1025.920044,1040.459961,1040.459961,4.313310e+09
2009-10-06,1042.020020,1060.550049,1042.020020,1054.719971,1054.719971,5.029840e+09
2009-10-07,1053.650024,1058.020020,1050.099976,1057.579956,1057.579956,4.238220e+09
2009-10-08,1060.030029,1070.670044,1060.030029,1065.479980,1065.479980,4.988400e+09
2009-10-09,1065.280029,1071.510010,1063.000000,1071.489990,1071.489990,3.763780e+09
2009-10-10,1065.280029,1071.510010,1063.000000,1071.489990,1071.489990,3.763780e+09
2009-10-11,1065.280029,1071.510010,1063.000000,1071.489990,1071.489990,3.763780e+09
2009-10-12,1071.630005,1079.459961,1071.630005,1076.189941,1076.189941,3.710430e+09
2009-10-13,1074.959961,1075.300049,1066.709961,1073.189941,1073.189941,4.320480e+09
2009-10-14,1078.680054,1093.170044,1078.680054,1092.020020,1092.020020,5.406420e+09


In [32]:
test_data_full.head(30)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Open,High,Low,Close,Adj Close,Volume
2021-10-01,4317.160156,4375.189941,4288.520020,4357.040039,4357.040039,4.010370e+09
2021-10-02,4317.160156,4375.189941,4288.520020,4357.040039,4357.040039,4.010370e+09
2021-10-03,4317.160156,4375.189941,4288.520020,4357.040039,4357.040039,4.010370e+09
2021-10-04,4348.839844,4355.509766,4278.939941,4300.459961,4300.459961,4.307870e+09
2021-10-05,4309.870117,4369.229980,4309.870117,4345.720215,4345.720215,3.902890e+09
2021-10-06,4319.569824,4365.569824,4290.490234,4363.549805,4363.549805,4.009630e+09
2021-10-07,4383.729980,4429.970215,4383.729980,4399.759766,4399.759766,3.843740e+09
2021-10-08,4406.509766,4412.020020,4386.220215,4391.339844,4391.339844,3.280160e+09
2021-10-09,4406.509766,4412.020020,4386.220215,4391.339844,4391.339844,3.280160e+09
2021-10-10,4406.509766,4412.020020,4386.220215,4391.339844,4391.339844,3.280160e+09


In [38]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_full['Volume_scaled'] = scaler.fit_transform(data_full[['Volume']])
train_data_full['Volume_scaled'] = scaler.fit_transform(train_data_full[['Volume']])
test_data_full['Volume_scaled'] = scaler.fit_transform(test_data_full[['Volume']])

In [39]:
data_full

,Open,High,Low,Close,Adj Close,Volume,Volume_scaled
2009-10-05,1026.869995,1042.579956,1025.920044,1040.459961,1040.459961,4.313310e+09,0.342789
2009-10-06,1042.020020,1060.550049,1042.020020,1054.719971,1054.719971,5.029840e+09,0.417484
2009-10-07,1053.650024,1058.020020,1050.099976,1057.579956,1057.579956,4.238220e+09,0.334961
2009-10-08,1060.030029,1070.670044,1060.030029,1065.479980,1065.479980,4.988400e+09,0.413164
2009-10-09,1065.280029,1071.510010,1063.000000,1071.489990,1071.489990,3.763780e+09,0.285503
...,...,...,...,...,...,...,...
2024-09-28,5755.359863,5763.779785,5727.339844,5738.169922,5738.169922,3.898530e+09,0.299550
2024-09-29,5755.359863,5763.779785,5727.339844,5738.169922,5738.169922,3.898530e+09,0.299550
2024-09-30,5726.520020,5765.140137,5703.529785,5762.479980,5762.479980,4.425730e+09,0.354508
2024-10-01,5757.729980,5757.729980,5681.279785,5708.750000,5708.750000,4.025180e+09,0.312753


In [40]:
train_data_full

,Open,High,Low,Close,Adj Close,Volume,Volume_scaled
2009-10-05,1026.869995,1042.579956,1025.920044,1040.459961,1040.459961,4.313310e+09,0.342789
2009-10-06,1042.020020,1060.550049,1042.020020,1054.719971,1054.719971,5.029840e+09,0.417484
2009-10-07,1053.650024,1058.020020,1050.099976,1057.579956,1057.579956,4.238220e+09,0.334961
2009-10-08,1060.030029,1070.670044,1060.030029,1065.479980,1065.479980,4.988400e+09,0.413164
2009-10-09,1065.280029,1071.510010,1063.000000,1071.489990,1071.489990,3.763780e+09,0.285503
...,...,...,...,...,...,...,...
2021-09-26,4438.040039,4463.120117,4430.270020,4455.479980,4455.479980,3.384290e+09,0.245944
2021-09-27,4442.120117,4457.299805,4436.189941,4443.109863,4443.109863,4.058650e+09,0.316242
2021-09-28,4419.540039,4419.540039,4346.330078,4352.629883,4352.629883,4.416550e+09,0.353551
2021-09-29,4362.410156,4385.569824,4355.080078,4359.459961,4359.459961,3.712660e+09,0.280174


In [41]:
test_data_full

,Open,High,Low,Close,Adj Close,Volume,Volume_scaled
2021-10-01,4317.160156,4375.189941,4288.520020,4357.040039,4357.040039,4.010370e+09,0.307315
2021-10-02,4317.160156,4375.189941,4288.520020,4357.040039,4357.040039,4.010370e+09,0.307315
2021-10-03,4317.160156,4375.189941,4288.520020,4357.040039,4357.040039,4.010370e+09,0.307315
2021-10-04,4348.839844,4355.509766,4278.939941,4300.459961,4300.459961,4.307870e+09,0.345878
2021-10-05,4309.870117,4369.229980,4309.870117,4345.720215,4345.720215,3.902890e+09,0.293384
...,...,...,...,...,...,...,...
2024-09-28,5755.359863,5763.779785,5727.339844,5738.169922,5738.169922,3.898530e+09,0.292818
2024-09-29,5755.359863,5763.779785,5727.339844,5738.169922,5738.169922,3.898530e+09,0.292818
2024-09-30,5726.520020,5765.140137,5703.529785,5762.479980,5762.479980,4.425730e+09,0.361155
2024-10-01,5757.729980,5757.729980,5681.279785,5708.750000,5708.750000,4.025180e+09,0.309235


In [11]:
data_scaled = data_full[['Close', 'Volume_scaled']].values

In [9]:
!pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 12.5 MB/s eta 0:00:00


The Machine Learning Model crashed when divided by 0 (current-price) occurred. This occurred because I originally scaled the close prices with the volume.


*   Do not scale the Close price. Keep prices in their raw form when calculating portfolio values and making buy/sell decisions.
*   Scale other features (e.g., Volume, technical indicators) that benefit from normalization.


*   Add checks for zero or near-zero values in your trading logic to prevent issues with calculations.








In [17]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class TradingEnv(gym.Env):
    def __init__(self, data, initial_balance=50000):
        super(TradingEnv, self).__init__()

        self.data = data
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.shares_held = 0
        self.portfolio_value = self.balance

        self.action_space = spaces.Discrete(3)  # Buy, Hold, Sell
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)

    def reset(self, seed=None, return_info=False, options=None):
        # Set the seed if provided
        super().reset(seed=seed)

        self.current_step = 0
        self.balance = self.initial_balance
        self.share_held = 0
        self.portfolio_value = self.balance
        self.previous_portfolio_value = self.portfolio_value # Track Portfolio value from the previous step

        # Return the initial observation and an optional info dict (empty for now)
        if return_info:
            return self.data[self.current_step], {}
        else:
            return self.data[self.current_step]


    def step(self, action):
        terminated = False
        truncated = False

        # Action: 0 = Hold, 1 = Buy, 2 = Sell
        current_price = self.data[self.current_step, 0] # Assuming 'Close' price is the 1st column

        # Action Logic
        if action == 1: # Buy
            if self.balance > 0: # Buy if you have cash available
                if current_price > 0: # Price is not zero to avoid division
                    self.shares_held = self.balance / current_price
                    self.balance = 0 # Cash becomes 0 after buying
                else:
                    print(f"Warning: Current price is zero at step {self.current_step}. Skipping buy action.")
            else:
                print(f"Warning: Balance is zero at step {self.current_step}. Cannot buy.")
        elif action == 2: # Sell
            if self.shares_held > 0: # Sell if you hold shares
                self.balance = self.shares_held * current_price
                self.shares_held = 0 # No more shares held after selling

        # Update Portfolio Value
        self.portfolio_value = self.balance + self.shares_held * current_price

        # Ensure Portfolio Value is not infinite
        if np.isinf(self.portfolio_value):
            print(f"Warning: Portfolio value is infinite at step {self.current_step}.")
            self.portfolio_value = self.initial_balance

        # Reward is based on the portfolio value increase
        reward = self.portfolio_value - self.previous_portfolio_value

        # Debugging: Print out step details
        print(f"Step: {self.current_step}, Action: {action}, Current Price: {current_price}")
        print(f"Shares Held: {self.shares_held}, Cash Balance: {self.balance}")
        print(f"Previous Portfolio Value: {self.previous_portfolio_value}, Current Portfolio Value: {self.portfolio_value}")
        print(f"Reward: {reward}\n")


        # Update the previous portfolio value for the next step
        self.previous_portfolio_value = self.portfolio_value

        # Move to the next step
        self.current_step += 1

        # End of data
        if self.current_step >= len(self.data) - 1:
            truncated = True

        return self.data[self.current_step], reward, terminated, truncated, {}

    def render(self):
        # This can be customized to display the portfolio performance over time
        print(f'Step: {self.current_step}')
        print(f'Portfolio Value: {self.portfolio_value}')
        print(f'Shares Held: {self.shares_held}')
        print(f'Cash Balance: {self.balance}')

### Update Buy and Sell Logic to control the amount of shares bought

### Fine-tune the action logic, reward system, and handling of special cases (like running out of data or specific conditions for termination)

In [10]:
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 3.6 MB/s eta 0:00:00


In [16]:
volume_column = data_scaled[:,1]
zero_indices = np.where(volume_column == 0)
print("Indices of zeros in Volume column:", zero_indices)

Indices of zeros in Volume column: (array([513]),)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
from stable_baselines3 import PPO

env = TradingEnv(data_scaled)
model = PPO("MlpPolicy", env, verbose=1, learning_rate=1e-5)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [15]:
print(np.isnan(data_scaled).sum())

0


In [20]:
model.learn(total_timesteps=100000)
model.save("ppo_trading_model")

rewards = []
for _ in range(100):
    obs = env.reset()
    done = False
    while not done:
        action, _states = model.predict(obs)
        obs, reward, done, truncated, info = env.step(action)

        # Combine done and truncated to determine if the episode is over
        done = done or truncated
        rewards.append(reward)




Streaming output truncated to the last 5000 lines.
Previous Portfolio Value: 71774.25005502159, Current Portfolio Value: 72816.21176404014
Reward: 1041.9617090185493

Step: 4511, Action: 0, Current Price: 4504.080078125
Shares Held: 15.873850377293868, Cash Balance: 0
Previous Portfolio Value: 72816.21176404014, Current Portfolio Value: 71497.09324750633
Reward: -1319.1185165338102

Step: 4512, Action: 2, Current Price: 4418.64013671875
Shares Held: 0, Cash Balance: 70140.83240137876
Previous Portfolio Value: 71497.09324750633, Current Portfolio Value: 70140.83240137876
Reward: -1356.2608461275668

Step: 4513, Action: 1, Current Price: 4418.64013671875
Shares Held: 15.87385037729387, Cash Balance: 0
Previous Portfolio Value: 70140.83240137876, Current Portfolio Value: 70140.83240137876
Reward: 0.0

Step: 4514, Action: 0, Current Price: 4418.64013671875
Shares Held: 15.87385037729387, Cash Balance: 0
Previous Portfolio Value: 70140.83240137876, Current Portfolio Value: 70140.83240137876

The RL Agent Learns a Policy from the PPO method. It adjusts it's parameters (weights) of the policy network over time.

At the end of training I will have a neural network that has learned a trading policy. It'll be able to observe market conditions (like price and volume) and make decisions ( buy, sell, hold ). Complex patterns that map from market data to actions.



1.   **Evaluate the model's performance** on new, unseen data to see how well it trades.
2.   **Analyze the model's behavior** by running simulations and examining how it racts to various market conditions.
3.   **Feature importance analysis** can sometimes give insights into which factors (volume, price movements) are more influential in the model's decisions.

**Risk Management** Need to explicitly train the model to incorporate risk management. It is aggressive right now just betting it all.

**To incorporate risk management**, include reward penalities for excessive losses or add features related to risk (like volatility) into the state observations.



In [ ]:
data_scaled

In [ ]:
data_scaled.shape

In [ ]:
if np.isnan(data_scaled).any():
  print("NaN values found in data_scaled")
else:
  print("No NaN values found in data_scaled")

No NaN values found in data_scaled


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
!git --version

git version 2.34.1


In [55]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


In [58]:
!pwd

/content


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
!ls

drive  ppo_trading_model.zip  Reinforcement_Learning_Trading  sample_data


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [60]:
!ls /content/drive/MyDrive/

'Colab Notebooks'


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [61]:
notebook_path = '/content/drive/MyDrive/Colab Notebooks/rl_trading.ipynb'

In [43]:
!git clone https://github.com/cody-mckeon/Reinforcement_Learning_Trading.git

Cloning into 'Reinforcement_Learning_Trading'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 15 (delta 6), reused 7 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 24.62 KiB | 12.31 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [57]:
!cp /content/drive/MyDrive/Colab Notebooks/rl_trading.ipynb /content/Reinforcement_Learning_Trading/

cp: cannot stat '/content/drive/MyDrive/rl_trading.ipynb': No such file or directory


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
